
Projet de Combinatoire et Énumération
====== 

[Sujet du projet](https://github.com/hivert/CombiFIIL/raw/master/SujetProjet/projet.pdf)

L'objectif de ce projet est d'implémenter un comptage d'objets combinatoire décrits par une grammaire quelconque, on utilisera pour ce faire les méthodes générales de produit cartésien et d'union disjointe vues en cours.



## Questions de cours

### Question 1

Pour les grammaires des arbres et des mots de Fibonnacci, donner dans un tableau pour
n = 0, . . . , 10 les réponses attendue pour la méthode count pour les 8 non terminaux
des mots de Fibonnacci et les 3 non terminaux des arbres binaires

_______________

|         |  0|  1|  2|  3|  4|  5|  6|  7|  8|  9| 10|
|:---     |---|---|---|---|---|---|---|---|---|---|---|
| Fib     |  1 | 2  | 3  | 5  |  8 | 13  | 21  | 34  | 55  |  89 | 144  |
| Cas1    |  0 | 2  | 3  | 5  | 8  | 13  | 21  | 34  | 55  | 89  | 144  |
| Cas2    |  0 | 1  | 1  | 2  | 3  | 5  | 8  | 13  | 21  | 34  | 55  |
| Vide    | 1  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| CasAu   | 0  | 1  | 2  | 3  | 5  | 8  | 13  | 21  | 34  | 55  | 89  |
| AtomA   |  0 | 1  | 0  | 0  | 0  | 0  | 0  | 0  |  0 | 0  | 0  |
| AtomB   | 0  | 1  | 0  | 0  | 0  | 0  | 0  | 0  |  0  | 0 | 0  |
| CasBAu  | 0  | 0  | 1  | 2  | 3  | 5  | 8  | 13  | 21  | 34  | 55  |

|         |  0|  1|  2|  3|  4|  5|  6|  7|  8|  9| 10|
|:---     |---|---|---|---|---|---|---|---|---|---|---|
| Tree    |  0 | 1  | 1  | 2  |  5 | 14  | 42  | 132  | 429  |  1430 | 4862  |
| Node    | 0  | 0  | 1  | 2  | 5  | 14  | 42  |  132 | 429  | 1430  | 4862  |
| Leaf    | 0  | 1  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |


### Question 2

Donner la grammaire de tous les mots sur l’alphabet A,B.

____________


### Question 3

Donner la grammaire des mots de Dyck, c’est-à-dire les mots sur l’alphabet {(, )} et
qui sont correctement parenthésés.

____________


### Question 4

Donner la grammaire de mots sur l’alphabet A,B qui n’ont pas trois lettres consécuti-
vement égales.

____________

### Question 5

Donner la grammaire des palindromes sur l’alphabet A, B, même question sur l’alphabet
A,B,C.

_____________


### Question 6

Donner la grammaire des mots sur l’alphabet A,B qui contiennent autant de lettres A
que de lettres B.

______________


### Question 7

Écrire une fonction qui vérifie qu’une grammaire est correcte, c’est-à-dire que chaque
non-terminal apparaissant dans une règle est bien défini par la grammaire.

____________

## Implémentation des classes de grammaire

Nous commençons par implémenter les différentes classes qui modéliseront la grammaire en Python, on suivra la hierarchie suivante:



- __AbstractRule__: Représente une abstraction des règles de la grammaire, en particulier elle implémentera un dictionnaire qui fera référence à la grammaire toute entière ainsi qu'une fonction __set_grammar__ qui permettra de modifier la grammaire (?)
- __ConstructorRule__: Règles de constructions qui répresentent les symboles non-terminaux et qui sont dans notre cas __UnionRule__ et __ProductRule__
    - ProductRule : Cette règle représente le produit cartésien de deux règles, elle hérite de la classe ConstructorRule
    - UnionRule : Il s'agit de l'union disjointe de deux règles, elle hérite également de la classe ConstructorRule
- __ConstantRule__: Règles constantes représentant les symboles terminaux
    - EpsilonRule: Règle représentant le symbole $\epsilon$
    - SingletonRule: Règle représentant un symbole terminal quelconque

In [2]:
from abc import ABCMeta, abstractmethod
from random import randint

class AbstractRule(metaclass=ABCMeta):

    def __init__(self, constant):
        self._grammar = {}
        self._constant = constant

    def _set_grammar(self, gram):
        self._grammar = gram
        
    @property
    def constant(self):
        return self._constant
    
    @abstractmethod
    def count(self, n):
        """count the numbers of objects of weight n that can be generated by the grammar"""
    
    @abstractmethod
    def list(self, n):
        """Enumerate objects of weight n that can be generated by the grammar and return them as a list"""
        
    @abstractmethod
    def unrank(self, n, rank):
        """Return object of size n and rank rank in the set generated by the grammar"""
        
    def random(self, n):
        """
        Return a random element of size n generated by the grammar
        """
        return self.unrank(n, randint(0, self.count(n)-1))

class ConstantRule(AbstractRule,metaclass=ABCMeta):
    def __init__(self, pyobj, valuation):
        super().__init__(True)
        self._object = pyobj
        self.__valuation = valuation
        
    @property
    def valuation(self):
        return self.__valuation


class ConstructorRule(AbstractRule,metaclass=ABCMeta):
    def __init__(self, value):
        super().__init__(False)
        self._valuation = float('inf')
        self._parameters = value

    @property
    def valuation(self):
        return self._valuation
    
    @property
    def rules(self):
        return tuple(map(lambda p : self._grammar[p], self._parameters))

    @abstractmethod
    def _calc_valuation(self):
        """update valuation"""


## Implémentation des classes héritières et algorithmique

Dans cette partie on implémente les classes Rule qui héritent de nos classes abstraites précedentes, c'est également là qu'on implémentera l'ensemble des fonctions (calcul de valuation, rank, unrank, count, ...)

In [21]:
from functools import lru_cache
CACHE_MAX_SIZE=50

class UnionRule(ConstructorRule):
    def __init__(self, rule1, rule2):
        super().__init__((rule1, rule2))
        
    def __repr__(self):
        return "UnionRule({},{})".format(self._parameters[0],self._parameters[1])
        
    def _calc_valuation(self):
        rule1, rule2 = self.rules
        valuation = min(rule1.valuation, rule2.valuation)
        self._valuation = valuation
        
    @lru_cache(maxsize=CACHE_MAX_SIZE)
    def count(self, n):
        rule1, rule2 = self.rules
        return rule1.count(n) + rule2.count(n)
    
    @lru_cache(maxsize=CACHE_MAX_SIZE)
    def list(self, n):
        rule1, rule2 = self.rules
        return rule1.list(n) + rule2.list(n)
    
    @lru_cache(maxsize=CACHE_MAX_SIZE)
    def unrank(self, n, rank):
        rule1, rule2 = self.rules
        rule1count = rule1.count(n)
        
        if rank >= rule1count:
            return rule2.unrank(n, rank - rule1count)
        else:
            return rule1.unrank(n, rank)
            

        
class ProductRule(ConstructorRule):
    def __init__(self, rule1, rule2, cons):
        super().__init__((rule1, rule2))
        self._cons = cons
    
    def __repr__(self):
        return "ProductRule({},{})".format(self._parameters[0],self._parameters[1])
        
    def _calc_valuation(self):
        rule1, rule2 = self.rules
        valuation = rule1.valuation + rule2.valuation
        self._valuation = valuation
    
    @lru_cache(maxsize=CACHE_MAX_SIZE)
    def count(self, n):
        rule1, rule2 = self.rules
        countsum=0
        for i in range(n + 1):
            k=n-i
            if rule1.valuation > i or rule2.valuation > k: 
                continue #Si cette condition est satisfaite alors
                         # rule1.count(i)*rule2.count(k) = 0 donc on passe
            countsum += rule1.count(i) * rule2.count(k)
        return countsum
    
    @lru_cache(maxsize=CACHE_MAX_SIZE)
    def list(self, n):
        rule1, rule2 = self.rules
        res=[]
        
        for i in range(n+1):
            k=n-i
            if rule1.valuation > i or rule2.valuation > k:
                continue
            for el1 in rule1.list(i):
                for el2 in rule2.list(k):
                    res.append(self._cons(el1,el2))
        return res
    
    
    @lru_cache(maxsize=CACHE_MAX_SIZE)
    def unrank(self, n, rank):
        if rank >= self.count(n):
            raise ValueError("Error called unrank with a too big rank")
            
        rule1, rule2 = self.rules 
        count, prec_count, i = 0, 0, 0
        #on compte les éléments de la grammaire pour chaque i jusqu'a 
        #ce qu'on en aie plus que le rang souhaité 
        while count <= rank:
            k = n - i
            prec_count=count
            count += rule1.count(i) * rule2.count(k)
            i += 1    
        i-=1
        j = rank - prec_count #le rang j est exactement le rang souhaité
        k = rule2.count(n-i)  # moins l'avant dernier comptage de notre boucle
        q, r = j // k, j % k
        
        return self._cons(rule1.unrank(i, q), rule2.unrank(n-i, r))
    
class EpsilonRule(ConstantRule):
    def __init__(self,obj):
        super().__init__(obj, 0)
        
    def __repr__(self):
        return "EpsilonRule({})".format(self._object)
        
    def count(self, n):
        return 1 if n==0 else 0
    
    def list(self, n):
        return [self._object] if n==0 else []
    
    def unrank(self, n, rank):
        if n==0 and rank==0:
            return self._object
        raise ValueError("Called unrank on EpsilonRule with wrong args{}".format(n,rank))
        
class SingletonRule(ConstantRule):
    def __init__(self,obj):
        super().__init__(obj, 1)
        
    def __repr__(self):
        return "SingletonRule({})".format(self._object)
        
    def count(self, n):
        return 1 if n==1 else 0
    
    def list(self, n):
        return [self._object] if n==1 else []
    
    def unrank(self, n, rank):
        if n==1 and rank==0:
            return self._object
        raise ValueError("Called unrank on SingletonRule constant with wrong args {}".format((n,rank)))

In [22]:
def init_grammar(gram):
    
    for rule in gram.values():
        if (not rule.constant):
            for pr in rule._parameters :
                if pr not in gram:
                    raise ValueError("A rule of the grammar ("+pr+") does not exist")
                
        rule._set_grammar(gram)
        
    valuation = [rule.valuation for rule in gram.values()]
    valuation_old = [float('inf') for _ in valuation]

    #algorithme de point fixe
    while (valuation != valuation_old):
        valuation_old = valuation
        for k, v in gram.items():
            if not v.constant:
                v._calc_valuation()
        valuation = [rule.valuation for rule in gram.values()]
        
    if float('inf') in valuation:
        inf_rules = [ k for k, v in gram.items() if v.valuation == float('inf')]
        raise ValueError("The grammar is incorrect, there is a rule that yields no terminal : {}".format(inf_rules))
                
    print("Valuation = {}".format({k : v.valuation for k,v in gram.items()}))
        
        

## Grammaires

Dans ce bloc on aura les différentes grammaires qui seront utilisées pour tester nos classes et méthodes précedentes.

In [23]:
from binarytree import BinaryTree, Leaf

stringsum = lambda a,b : a + b

#Grammaire de a^n . b^n pour n pair
grammarTest = {
    "Axiom" : UnionRule("Vide","Axiom1"),
    "Axiom1" : ProductRule("SingA","IntermSingB",stringsum),
    "Vide" : EpsilonRule(""),
    "SingA" : SingletonRule("A"),
    "SingB" : SingletonRule("B"),
    "IntermSingB" : ProductRule("Axiom","SingB",stringsum)
}
init_grammar(grammarTest)

#Grammaire des mots de fibonnaci
fiboGram = {    "Fib"    : UnionRule("Vide", "Cas1"),
                "Cas1"   : UnionRule("CasAu", "Cas2"),
                "Cas2"   : UnionRule("AtomB", "CasBAu"),
                "Vide"   : EpsilonRule(""),
                "CasAu"  : ProductRule("AtomA", "Fib", stringsum),
                "AtomA"  : SingletonRule("A"),
                "AtomB"  : SingletonRule("B"),
                "CasBAu" : ProductRule("AtomB", "CasAu", stringsum)}
init_grammar(fiboGram)

#Grammaire des arbres binaires
treeGram = {
    "Tree": UnionRule("Node", "Leaf"),
    "Node": ProductRule("Tree", "Tree", lambda a, b: BinaryTree([a, b])),
    "Leaf": SingletonRule(Leaf)
}
init_grammar(treeGram)

Valuation = {'Axiom': 0, 'Axiom1': 2, 'Vide': 0, 'SingA': 1, 'SingB': 1, 'IntermSingB': 1}
Valuation = {'Fib': 0, 'Cas1': 1, 'Cas2': 1, 'Vide': 0, 'CasAu': 1, 'AtomA': 1, 'AtomB': 1, 'CasBAu': 2}
Valuation = {'Tree': 1, 'Node': 2, 'Leaf': 1}


## Tests fonctionnels

Spécification minimale pour les classes Rule,

- self.unrank(n, k) == self.list(n)[k]
- self.count(n) == len(self.list(n))
- self.unrank(n, k) must fail for all k >= n


In [24]:
def runTest(gram,max_size=15):
    
    def testUnrank(n_samples, size):
        """
        Run tests on the unrank methods using n_samples random samples from a random size set.
        """
        
        #print(size)
        
        # gram.unrank(n, gram.count(m)) must fail for all m >= n
        setsize = gram.count(size)
        
        if setsize==0:
            print("This rule {} yields no elements for n={}, skipping test...".format(gram,size))
            return
        
        try:
            gram.unrank(size, setsize)
            assert(False)
        except ValueError:
            assert(True)
            
        # gram.unrank(n, k) == gram.list(n)[k]
        
        n_samples = min(n_samples,size)
        
        samples = [randint(0,setsize-1) for _ in range(n_samples)] #tire des élements de la grammaire
                                                                   #de manière aléatoire
        gramlist = gram.list(size)
        
        #print(samples)
        for sample in samples:
            assert(gram.unrank(size, sample) == gramlist[sample])
            
    def testCountList(size):
        """
        Same as above but run test on count and list
        """
        
        assert(gram.count(size) == len(gram.list(size)))
        

    
    for n in range(max_size):
        testUnrank(5,n)
    print("testUnrank: passed")
    for n in range(max_size):
        testCountList(n)
    print("testCountList: passed")

In [25]:
runTest(fiboGram["Fib"],25)

testUnrank: passed
testCountList: passed


In [26]:
runTest(treeGram["Tree"],10)

This rule UnionRule(Node,Leaf) yields no elements for n=0, skipping test...
testUnrank: passed
testCountList: passed


In [30]:
runTest(grammarTest["Axiom"],30)

This rule UnionRule(Vide,Axiom1) yields no elements for n=1, skipping test...
This rule UnionRule(Vide,Axiom1) yields no elements for n=3, skipping test...
This rule UnionRule(Vide,Axiom1) yields no elements for n=5, skipping test...
This rule UnionRule(Vide,Axiom1) yields no elements for n=7, skipping test...
This rule UnionRule(Vide,Axiom1) yields no elements for n=9, skipping test...
This rule UnionRule(Vide,Axiom1) yields no elements for n=11, skipping test...
This rule UnionRule(Vide,Axiom1) yields no elements for n=13, skipping test...
This rule UnionRule(Vide,Axiom1) yields no elements for n=15, skipping test...
This rule UnionRule(Vide,Axiom1) yields no elements for n=17, skipping test...
This rule UnionRule(Vide,Axiom1) yields no elements for n=19, skipping test...
This rule UnionRule(Vide,Axiom1) yields no elements for n=21, skipping test...
This rule UnionRule(Vide,Axiom1) yields no elements for n=23, skipping test...
This rule UnionRule(Vide,Axiom1) yields no elements for n

## Sandbox

In [28]:
N=8
Gram=treeGram["Tree"]
print("Set has size {}".format(Gram.count(N)))
[print(Gram.unrank(N,n)) for n in range(Gram.count(N))]
None

Set has size 429
(leaf, (leaf, (leaf, (leaf, (leaf, (leaf, (leaf, leaf)))))))
(leaf, (leaf, (leaf, (leaf, (leaf, ((leaf, leaf), leaf))))))
(leaf, (leaf, (leaf, (leaf, ((leaf, leaf), (leaf, leaf))))))
(leaf, (leaf, (leaf, (leaf, ((leaf, (leaf, leaf)), leaf)))))
(leaf, (leaf, (leaf, (leaf, (((leaf, leaf), leaf), leaf)))))
(leaf, (leaf, (leaf, ((leaf, leaf), (leaf, (leaf, leaf))))))
(leaf, (leaf, (leaf, ((leaf, leaf), ((leaf, leaf), leaf)))))
(leaf, (leaf, (leaf, ((leaf, (leaf, leaf)), (leaf, leaf)))))
(leaf, (leaf, (leaf, (((leaf, leaf), leaf), (leaf, leaf)))))
(leaf, (leaf, (leaf, ((leaf, (leaf, (leaf, leaf))), leaf))))
(leaf, (leaf, (leaf, ((leaf, ((leaf, leaf), leaf)), leaf))))
(leaf, (leaf, (leaf, (((leaf, leaf), (leaf, leaf)), leaf))))
(leaf, (leaf, (leaf, (((leaf, (leaf, leaf)), leaf), leaf))))
(leaf, (leaf, (leaf, ((((leaf, leaf), leaf), leaf), leaf))))
(leaf, (leaf, ((leaf, leaf), (leaf, (leaf, (leaf, leaf))))))
(leaf, (leaf, ((leaf, leaf), (leaf, ((leaf, leaf), leaf)))))
(leaf, 

In [ ]:
grammarTest["Axiom"].count(100)

In [ ]:
[grammarTest["Axiom"].list(n) for n in range(20)]

In [ ]:
stringsum = lambda a,b : a + b
fiboGram = {    "Fib"    : UnionRule("Vide", "Cas1"),
                "Cas1"   : UnionRule("CasAu", "Cas2"),
                "Cas2"   : UnionRule("AtomB", "CasBAu"),
                "Vide"   : EpsilonRule(""),
                "CasAu"  : ProductRule("AtomA", "Fib", stringsum),
                "AtomA"  : SingletonRule("A"),
                "AtomB"  : SingletonRule("B"),
                "CasBAu" : ProductRule("AtomB", "CasAu", stringsum)}
init_grammar(fiboGram)

In [77]:
fiboGram["Fib"].random(10)

'ABAAAAABAA'

In [78]:
fiboGram["Fib"].list(10)

['AAAAAAAAAA',
 'AAAAAAAAAB',
 'AAAAAAAABA',
 'AAAAAAABAA',
 'AAAAAAABAB',
 'AAAAAABAAA',
 'AAAAAABAAB',
 'AAAAAABABA',
 'AAAAABAAAA',
 'AAAAABAAAB',
 'AAAAABAABA',
 'AAAAABABAA',
 'AAAAABABAB',
 'AAAABAAAAA',
 'AAAABAAAAB',
 'AAAABAAABA',
 'AAAABAABAA',
 'AAAABAABAB',
 'AAAABABAAA',
 'AAAABABAAB',
 'AAAABABABA',
 'AAABAAAAAA',
 'AAABAAAAAB',
 'AAABAAAABA',
 'AAABAAABAA',
 'AAABAAABAB',
 'AAABAABAAA',
 'AAABAABAAB',
 'AAABAABABA',
 'AAABABAAAA',
 'AAABABAAAB',
 'AAABABAABA',
 'AAABABABAA',
 'AAABABABAB',
 'AABAAAAAAA',
 'AABAAAAAAB',
 'AABAAAAABA',
 'AABAAAABAA',
 'AABAAAABAB',
 'AABAAABAAA',
 'AABAAABAAB',
 'AABAAABABA',
 'AABAABAAAA',
 'AABAABAAAB',
 'AABAABAABA',
 'AABAABABAA',
 'AABAABABAB',
 'AABABAAAAA',
 'AABABAAAAB',
 'AABABAAABA',
 'AABABAABAA',
 'AABABAABAB',
 'AABABABAAA',
 'AABABABAAB',
 'AABABABABA',
 'ABAAAAAAAA',
 'ABAAAAAAAB',
 'ABAAAAAABA',
 'ABAAAAABAA',
 'ABAAAAABAB',
 'ABAAAABAAA',
 'ABAAAABAAB',
 'ABAAAABABA',
 'ABAAABAAAA',
 'ABAAABAAAB',
 'ABAAABAABA',
 'ABAAABAB

In [ ]:
fiboGram["Fib"].count(100)

In [ ]:
fiboGram["Fib"].unrank(10,10)

In [ ]:
fiboGram["Fib"].unrank(10,fiboGram["Fib"].count(10))

In [ ]:
treeGram = {
    "Tree": UnionRule("Node", "Leaf"),
    "Node": ProductRule("Tree", "Tree", lambda a, b: BinaryTree([a, b])),
    "Leaf": SingletonRule(leaf)
}

In [ ]:
init_grammar(treeGram)

In [ ]:
treeGram["Tree"].list(5)